# 02 - State 管理實作

## 📚 學習目標

- ✅ 理解 State 的更新機制
- ✅ 使用 Annotated 實現狀態累加
- ✅ 追蹤和除錯 State 變化
- ✅ 設計複雜的 State 結構

## ⏰ 預計時間: 40 分鐘

In [1]:
# 環境設定
from typing import TypedDict, Annotated
from operator import add
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
print("✅ 環境設定完成")

✅ 環境設定完成


## Part 1: State 更新機制

### 1.1 覆蓋更新 (Replace)

In [2]:
class SimpleState(TypedDict):
    """簡單的狀態 - 預設是覆蓋更新"""
    counter: int
    name: str

def increment(state: SimpleState) -> SimpleState:
    """遞增計數器"""
    return {
        "counter": state["counter"] + 1,
        "name": "updated"
    }

# 測試
workflow = StateGraph(SimpleState)
workflow.add_node("increment", increment)
workflow.set_entry_point("increment")
workflow.add_edge("increment", END)
app = workflow.compile()

result = app.invoke({"counter": 0, "name": "original"})
print(f"執行前: counter=0, name='original'")
print(f"執行後: counter={result['counter']}, name='{result['name']}'")
print("\n📌 覆蓋更新: 新值完全取代舊值")

執行前: counter=0, name='original'
執行後: counter=1, name='updated'

📌 覆蓋更新: 新值完全取代舊值


### 1.2 累加更新 (Add with Annotated)

In [3]:
class AccumulativeState(TypedDict):
    """累加型狀態"""
    messages: Annotated[list[str], add]  # 累加
    count: int                            # 覆蓋

def add_message(state: AccumulativeState) -> AccumulativeState:
    """添加訊息"""
    return {
        "messages": [f"訊息 {state['count'] + 1}"],
        "count": state['count'] + 1
    }

# 建立 Agent
workflow = StateGraph(AccumulativeState)
workflow.add_node("add_msg", add_message)
workflow.set_entry_point("add_msg")

# 循環 3 次
def should_continue(state):
    return "add" if state["count"] < 3 else "end"

workflow.add_conditional_edges(
    "add_msg",
    should_continue,
    {"add": "add_msg", "end": END}
)

app = workflow.compile()
result = app.invoke({"messages": [], "count": 0})

print("累加結果:")
print(f"訊息列表: {result['messages']}")
print(f"計數: {result['count']}")
print("\n📌 累加更新: 新值加到現有列表後面")

累加結果:
訊息列表: ['訊息 1', '訊息 2', '訊息 3']
計數: 3

📌 累加更新: 新值加到現有列表後面


## Part 2: 複雜 State 設計

### 實戰案例: 客戶服務對話狀態

In [4]:
from datetime import datetime
from typing import Optional

class ConversationState(TypedDict):
    """完整的對話狀態設計"""
    # 基本資訊
    session_id: str
    user_id: str
    created_at: str
    
    # 對話內容 (累加)
    messages: Annotated[list[dict], add]
    
    # 分析結果 (覆蓋)
    current_intent: str
    sentiment: str
    
    # 處理狀態 (覆蓋)
    is_resolved: bool
    requires_human: bool
    
    # 追蹤資訊 (累加)
    execution_log: Annotated[list[str], add]

print("✅ 複雜 State 定義完成")
print(f"欄位數量: {len(ConversationState.__annotations__)}")
print(f"累加欄位: messages, execution_log")
print(f"覆蓋欄位: 其他所有欄位")

✅ 複雜 State 定義完成
欄位數量: 9
累加欄位: messages, execution_log
覆蓋欄位: 其他所有欄位


### 使用複雜 State

In [5]:
def process_message(state: ConversationState) -> ConversationState:
    """處理使用者訊息"""
    llm = ChatOpenAI(model="gpt-4")
    
    # 取得最後一則訊息
    last_msg = state["messages"][-1]["content"] if state["messages"] else ""
    
    # 分析意圖
    intent_prompt = f"簡短判斷意圖(查詢/投訴/諮詢): {last_msg}"
    intent = llm.invoke(intent_prompt).content
    
    # 分析情感
    sentiment_prompt = f"判斷情感(正面/負面/中性): {last_msg}"
    sentiment = llm.invoke(sentiment_prompt).content
    
    # 生成回應
    response = llm.invoke(f"專業回應: {last_msg}").content
    
    return {
        "messages": [{"role": "assistant", "content": response}],
        "current_intent": intent.strip(),
        "sentiment": sentiment.strip(),
        "execution_log": [f"處理訊息於 {datetime.now().strftime('%H:%M:%S')}"]
    }

# 建立 Agent
workflow = StateGraph(ConversationState)
workflow.add_node("process", process_message)
workflow.set_entry_point("process")
workflow.add_edge("process", END)
conversation_agent = workflow.compile()

# 測試
result = conversation_agent.invoke({
    "session_id": "S001",
    "user_id": "U001",
    "created_at": datetime.now().isoformat(),
    "messages": [{"role": "user", "content": "我想查詢保單資訊"}],
    "is_resolved": False,
    "requires_human": False,
    "execution_log": []
})

print("\n對話結果:")
print(f"意圖: {result['current_intent']}")
print(f"情感: {result['sentiment']}")
print(f"訊息數: {len(result['messages'])}")
print(f"執行日誌: {result['execution_log']}")


對話結果:
意圖: 查詢
情感: 中性
訊息數: 2
執行日誌: ['處理訊息於 01:03:58']


## Part 3: State 追蹤與除錯

In [6]:
class DebugState(TypedDict):
    """帶除錯資訊的狀態"""
    data: str
    step_history: Annotated[list[dict], add]

def debug_node(node_name: str):
    """建立帶除錯的節點"""
    def wrapper(state: DebugState) -> DebugState:
        print(f"\n🔍 執行節點: {node_name}")
        print(f"   輸入 data: {state['data']}")
        
        # 處理
        new_data = state['data'] + f" -> {node_name}"
        
        print(f"   輸出 data: {new_data}")
        
        return {
            "data": new_data,
            "step_history": [{
                "node": node_name,
                "input": state['data'],
                "output": new_data,
                "timestamp": datetime.now().isoformat()
            }]
        }
    return wrapper

# 建立除錯 Agent
workflow = StateGraph(DebugState)
workflow.add_node("step1", debug_node("步驟1"))
workflow.add_node("step2", debug_node("步驟2"))
workflow.add_node("step3", debug_node("步驟3"))

workflow.set_entry_point("step1")
workflow.add_edge("step1", "step2")
workflow.add_edge("step2", "step3")
workflow.add_edge("step3", END)

debug_agent = workflow.compile()

# 執行並追蹤
print("\n" + "="*50)
print("開始追蹤執行過程")
print("="*50)

result = debug_agent.invoke({
    "data": "初始資料",
    "step_history": []
})

print("\n" + "="*50)
print("執行歷史:")
print("="*50)
for i, step in enumerate(result['step_history'], 1):
    print(f"\n步驟 {i}: {step['node']}")
    print(f"  輸入: {step['input']}")
    print(f"  輸出: {step['output']}")
    print(f"  時間: {step['timestamp']}")


開始追蹤執行過程

🔍 執行節點: 步驟1
   輸入 data: 初始資料
   輸出 data: 初始資料 -> 步驟1

🔍 執行節點: 步驟2
   輸入 data: 初始資料 -> 步驟1
   輸出 data: 初始資料 -> 步驟1 -> 步驟2

🔍 執行節點: 步驟3
   輸入 data: 初始資料 -> 步驟1 -> 步驟2
   輸出 data: 初始資料 -> 步驟1 -> 步驟2 -> 步驟3

執行歷史:

步驟 1: 步驟1
  輸入: 初始資料
  輸出: 初始資料 -> 步驟1
  時間: 2025-11-15T01:07:21.004373

步驟 2: 步驟2
  輸入: 初始資料 -> 步驟1
  輸出: 初始資料 -> 步驟1 -> 步驟2
  時間: 2025-11-15T01:07:21.006442

步驟 3: 步驟3
  輸入: 初始資料 -> 步驟1 -> 步驟2
  輸出: 初始資料 -> 步驟1 -> 步驟2 -> 步驟3
  時間: 2025-11-15T01:07:21.006996


## 🎯 練習題

### 練習 1: 購物車狀態管理

In [7]:
# 定義購物車狀態
class ShoppingCartState(TypedDict):
    # TODO: 設計狀態欄位
    # 提示: 需要商品列表(累加)、總金額(覆蓋)、優惠碼等
    pass

# TODO: 實作添加商品、計算總額、套用折扣的節點

# TODO: 建立購物車 Agent

### 練習 2: 審批流程狀態

In [8]:
# 設計一個多級審批系統的狀態
class ApprovalState(TypedDict):
    # TODO: 定義狀態
    # 提示: 審批歷史(累加)、當前狀態(覆蓋)、審批人資訊等
    pass

## ✅ 解答參考

In [9]:
# 練習 1 解答
class ShoppingCartStateSolution(TypedDict):
    items: Annotated[list[dict], add]  # 商品列表
    total: float                        # 總金額
    discount_code: Optional[str]        # 優惠碼
    discount_amount: float              # 折扣金額
    final_total: float                  # 最終金額

def add_item(state, item):
    """添加商品"""
    return {
        "items": [item],
        "total": state.get("total", 0) + item["price"]
    }

def apply_discount(state):
    """套用折扣"""
    discount = 0
    if state.get("discount_code") == "SAVE10":
        discount = state["total"] * 0.1
    
    return {
        "discount_amount": discount,
        "final_total": state["total"] - discount
    }

print("✅ 購物車狀態設計完成")

✅ 購物車狀態設計完成


## 📝 重點回顧

### State 更新機制
- **覆蓋**: 預設行為,新值取代舊值
- **累加**: 使用 `Annotated[type, add]`,新值加到舊值後面

### 設計原則
1. 對話/日誌類欄位使用累加
2. 狀態/配置類欄位使用覆蓋
3. 使用 Optional 標記可選欄位
4. 添加追蹤欄位便於除錯

### 除錯技巧
- 使用 step_history 追蹤執行過程
- 印出每個節點的輸入輸出
- 記錄時間戳協助效能分析

### 下一步
繼續前往 `03-conditional-edges.ipynb` 學習條件邊的使用 👉